<a href="https://colab.research.google.com/github/Chanwoong1/recommendation-system-study/blob/main/tf_idf_with_kaggle_movie_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# kaggle dataset load

In [1]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"chanwoongjeong","key":"9ee1bdf4f921a149a3adec1b35ec64fa"}'}

In [2]:
ls -1ha kaggle.json

kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json
# 내가 참가한 대회 리스트 확인
!kaggle competitions list

ref                                            deadline             category            reward  teamCount  userHasEntered  
---------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
contradictory-my-dear-watson                   2030-07-01 23:59:00  Getting Started     Prizes         90           False  
gan-getting-started                            2030-07-01 23:59:00  Getting Started     Prizes         90           False  
store-sales-time-series-forecasting            2030-06-30 23:59:00  Getting Started  Knowledge        958           False  
tpu-getting-started                            2030-06-03 23:59:00  Getting Started  Knowledge        150           False  
digit-recognizer                               2030-01-01 00:00:00  Getting Started  Knowledge       1726           False  
titanic                                        2030-01-01 00:00:00  Getting Started  Knowledge      14633            True  
house-pr

In [4]:
# kaggle dataset load
! kaggle datasets download -d rounakbanik/the-movies-dataset

 95% 216M/228M [00:01<00:00, 136MB/s]
100% 228M/228M [00:01<00:00, 139MB/s]


In [5]:
!unzip the-movies-dataset.zip

Archive:  the-movies-dataset.zip
  inflating: credits.csv             
  inflating: keywords.csv            
  inflating: links.csv               
  inflating: links_small.csv         
  inflating: movies_metadata.csv     
  inflating: ratings.csv             
  inflating: ratings_small.csv       


# TF-IDF

In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
path = './'
data = pd.read_csv(path + 'movies_metadata.csv', low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [14]:
# 컬럼 확인 
data.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

overview 항목을 가지고 진행해본다

In [15]:
# 전처리 
# overview의 결측치가 있는 항목은 모두 제거 
data = data[data['overview'].notnull()].reset_index(drop=True)
data.shape

(44512, 24)

In [16]:
# 불용어 : 유의미하지 않은 단어 토큰을 제거 
# https://wikidocs.net/22530
tfidf = TfidfVectorizer(stop_words='english', max_features=20000) # 메모리 부족으로 데이터 줄여서 사용
data = data.loc[0:20000].reset_index(drop=True)

# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['overview'])
print(tfidf_matrix.shape)

(20001, 20000)


In [17]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [18]:
np.round(cosine_matrix, 4)

array([[1.    , 0.0161, 0.    , ..., 0.009 , 0.0184, 0.    ],
       [0.0161, 1.    , 0.0492, ..., 0.0061, 0.0084, 0.    ],
       [0.    , 0.0492, 1.    , ..., 0.    , 0.    , 0.    ],
       ...,
       [0.009 , 0.0061, 0.    , ..., 1.    , 0.0162, 0.    ],
       [0.0184, 0.0084, 0.    , ..., 0.0162, 1.    , 0.0211],
       [0.    , 0.    , 0.    , ..., 0.    , 0.0211, 1.    ]])

In [19]:
# movie title와 id를 매핑할 dictionary를 생성해준다. 
movie2id = {}
for i, c in enumerate(data['title']): movie2id[i] = c

# id와 movie title를 매핑할 dictionary를 생성해준다. 
id2movie = {}
for i, c in movie2id.items(): id2movie[c] = i

In [22]:
# Toy Story의 id 추출 
idx = id2movie['Toy Story'] # Toy Story : 0번 인덱스 
sim_scores = [(i, c) for i, c in enumerate(cosine_matrix[idx]) if i != idx] # 자기 자신을 제외한 영화들의 유사도 및 인덱스를 추출 
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse=True) # 유사도가 높은 순서대로 정렬 
sim_scores[0:10] # 상위 10개의 인덱스와 유사도를 추출 

[(15282, 0.5365942551545341),
 (2979, 0.4517509299198836),
 (10271, 0.3147028032478532),
 (8303, 0.20826951294595616),
 (1058, 0.18647595466742906),
 (1916, 0.17211113868192898),
 (11367, 0.17195688896142478),
 (6920, 0.15224062928617918),
 (17102, 0.1476210170507341),
 (3039, 0.14616894557243704)]

In [23]:
# 인덱스를 Movie Title로 변환 
sim_scores = [(movie2id[i], score) for i, score in sim_scores[0:10]]
sim_scores

[('Toy Story 3', 0.5365942551545341),
 ('Toy Story 2', 0.4517509299198836),
 ('The 40 Year Old Virgin', 0.3147028032478532),
 ('The Champ', 0.20826951294595616),
 ('Rebel Without a Cause', 0.18647595466742906),
 ('Condorman', 0.17211113868192898),
 ('For Your Consideration', 0.17195688896142478),
 ('Rivers and Tides', 0.15224062928617918),
 ('Group Sex', 0.1476210170507341),
 ('Man on the Moon', 0.14616894557243704)]